In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from functools import partial
import gc
import json
import os.path
import time

import datatable as dt
from joblib import delayed, parallel

In [ ]:
INPUT_DIR = '/kaggle/input/mlb-player-digital-engagement-forecasting/'

# train.csv

In [ ]:
%%time
train = dt.fread(os.path.join(INPUT_DIR, 'train_updated.csv')).to_pandas()
train

In [ ]:
def unpack_json(row: pd.Series, json_col: str) -> pd.DataFrame:
    try:
        json_dict = json.loads(row[json_col])
    except json.JSONDecodeError:
        return pd.DataFrame()
    json_df = pd.DataFrame(json_dict)
    json_df['date__'] = row.date
    return json_df

In [ ]:
%%time
since = time.time()
json_columns = train.drop(columns='date').columns.tolist()
for c in json_columns:
    unpack_func = partial(unpack_json, json_col=c)
    json_df = pd.concat((train[['date', c]].dropna().apply(unpack_func, axis=1)).tolist())
    print(f'Extract {json_df.shape[0]} rows from {c} ({time.time() - since:.5f} seconds passed)')
    json_df.to_csv(f'{c}.csv', index=False)
    print(f'Write "{c}.csv" ({time.time() - since:.5f} seconds passed)')
    del json_df
    gc.collect()